In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:

size1 = 12
size2 = 16
size3 = 24
drop_value = 0.1

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, size1, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(size1),
            nn.Dropout(drop_value),
            nn.Conv2d(size1, size1, 3, padding=1),
            #
            nn.ReLU(),
            nn.BatchNorm2d(size1),
            nn.Dropout(drop_value),
            nn.Conv2d(size1, size1, 3, stride=1, padding=1),
            #
            nn.ReLU(),
            nn.BatchNorm2d(size1),
            nn.Dropout(drop_value),
            nn.Conv2d(size1, size1, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(size1),
            nn.MaxPool2d(2, 2),
            nn.Dropout(drop_value)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(size1, size2, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(size2),
            nn.Dropout(drop_value),
            nn.Conv2d(size2, size2, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(size2),
            nn.Dropout(drop_value),
            nn.Conv2d(size2, size2, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(size2),
            nn.MaxPool2d(2, 2),
            nn.Dropout(drop_value)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(size2, size3, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(size3),
            nn.Dropout(drop_value),

            nn.Conv2d(size3, size3, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(size3),
            nn.Dropout(drop_value),

            nn.MaxPool2d(2, 2),
        )


        # OUTPUT BLOCK
        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=3)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=size3, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        x = self.gap(x)
        x = self.conv4(x)
        x = x.view(x.size(0), -1)

        x = F.log_softmax(x, dim=1)
        return x


In [3]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 28, 28]             120
              ReLU-2           [-1, 12, 28, 28]               0
       BatchNorm2d-3           [-1, 12, 28, 28]              24
           Dropout-4           [-1, 12, 28, 28]               0
            Conv2d-5           [-1, 12, 28, 28]           1,308
              ReLU-6           [-1, 12, 28, 28]               0
       BatchNorm2d-7           [-1, 12, 28, 28]              24
           Dropout-8           [-1, 12, 28, 28]               0
            Conv2d-9           [-1, 12, 28, 28]           1,308
             ReLU-10           [-1, 12, 28, 28]               0
      BatchNorm2d-11           [-1, 12, 28, 28]              24
          Dropout-12           [-1, 12, 28, 28]               0
           Conv2d-13           [-1, 12, 28, 28]           1,308
             ReLU-14           [-1, 12,

In [4]:


torch.manual_seed(1897)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
                        #  ensure that all images have the same size before being fed into a neural network
                        transforms.Resize((28, 28)),
                        transforms.RandomRotation((-15.0, 15.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:00<00:00, 82474144.81it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 23210518.07it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 37193009.78it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 10536796.88it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        # Update pbar-tqdm

        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

for epoch in range(1, 20):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    # Step the scheduler
    scheduler.step()

EPOCH: 1


Loss=0.06296411156654358 Batch_id=468 Accuracy=87.75: 100%|██████████| 469/469 [02:42<00:00,  2.88it/s]



Test set: Average loss: 0.0580, Accuracy: 9818/10000 (98.18%)

EPOCH: 2


Loss=0.08837777376174927 Batch_id=468 Accuracy=97.07: 100%|██████████| 469/469 [02:39<00:00,  2.95it/s]



Test set: Average loss: 0.0474, Accuracy: 9857/10000 (98.57%)

EPOCH: 3


...


Test set: Average loss: 0.0166, Accuracy: 9950/10000 (99.50%)

EPOCH: 18


Loss=0.01692269742488861 Batch_id=468 Accuracy=98.71: 100%|██████████| 469/469 [02:35<00:00,  3.01it/s]



Test set: Average loss: 0.0168, Accuracy: 9948/10000 (99.48%)

EPOCH: 19


Loss=0.041301969438791275 Batch_id=468 Accuracy=98.69: 100%|██████████| 469/469 [02:35<00:00,  3.02it/s]



Test set: Average loss: 0.0173, Accuracy: 9946/10000 (99.46%)

